### Tratamento de CSV para Banco de Dados Relacional (MySQL)

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime

#### Modelagem de Dados
<img src="../sql/model/modelagem.png" width="750px">

#### Traduzindo as colunas

In [68]:
colunas =['id', 'id_ped', 'data_pedido', 'data_envio', 'modo_envio', 'id_cons', 'nome_consumidor', 'perfil_consumidor', 
          'pais', 'cidade', 'estado', 'codigo_postal', 'regiao', 'id_prod',  'categoria', 'sub_categoria', 'nome_produto', 'preco']

df_original = pd.read_csv("C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/data/sales.csv", sep = ",", names = colunas, header = 0, encoding = 'utf-8')

In [69]:
df_original.head()

,id,id_ped,data_pedido,data_envio,modo_envio,id_cons,nome_consumidor,perfil_consumidor,pais,cidade,estado,codigo_postal,regiao,id_prod,categoria,sub_categoria,nome_produto,preco
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


#### Criando os dataframes representantes de entidades do Banco de Dados

In [70]:
df_categoria = df_original[['categoria']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id', 'categoria': 'titulo'})
df_categoria['id'] = df_categoria['id'] + 1
df_categoria.head()

,id,titulo
0,1,Furniture
1,2,Office Supplies
2,3,Technology


In [71]:
df_sub_categoria = df_original[['sub_categoria', 'categoria']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id', 'sub_categoria': 'titulo'})
df_sub_categoria['id'] = df_sub_categoria['id'] + 1
df_sub_categoria.head()

,id,titulo,categoria
0,1,Bookcases,Furniture
1,2,Chairs,Furniture
2,3,Labels,Office Supplies
3,4,Tables,Furniture
4,5,Storage,Office Supplies


In [72]:
df_produto = df_original[['id_prod', 'nome_produto', 'sub_categoria']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id', 'nome_produto': 'nome'})
df_produto['id'] = df_produto['id'] + 1
df_produto.head()

,id,id_prod,nome,sub_categoria
0,1,FUR-BO-10001798,Bush Somerset Collection Bookcase,Bookcases
1,2,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Chairs
2,3,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Labels
3,4,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Tables
4,5,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Storage


In [73]:
df_endereco = df_original[['pais', 'cidade', 'estado', 'codigo_postal', 'regiao']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id'})
df_endereco['id'] = df_endereco['id'] + 1

df_endereco.head(5)

,id,pais,cidade,estado,codigo_postal,regiao
0,1,United States,Henderson,Kentucky,42420.0,South
1,2,United States,Los Angeles,California,90036.0,West
2,3,United States,Fort Lauderdale,Florida,33311.0,South
3,4,United States,Los Angeles,California,90032.0,West
4,5,United States,Concord,North Carolina,28027.0,South


In [75]:
df_transporte = df_original[['modo_envio']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id', 'modo_envio': 'titulo'})
df_transporte['id'] = df_transporte['id'] + 1

df_transporte.head()

,id,titulo
0,1,Second Class
1,2,Standard Class
2,3,First Class
3,4,Same Day


In [76]:
df_perfil = df_original[['perfil_consumidor']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id', 'perfil_consumidor': 'titulo'})
df_perfil['id'] = df_perfil['id'] + 1

df_perfil.head()

,id,titulo
0,1,Consumer
1,2,Corporate
2,3,Home Office


In [94]:
df_consumidor = df_original[['id_cons', 'nome_consumidor', 'perfil_consumidor']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id', 'nome_consumidor': 'nome'})
df_consumidor['id'] = df_consumidor['id'] + 1

df_consumidor.head()

,id,id_cons,nome,perfil_consumidor
0,1,CG-12520,Claire Gute,Consumer
1,2,DV-13045,Darrin Van Huff,Corporate
2,3,SO-20335,Sean O'Donnell,Consumer
3,4,BH-11710,Brosina Hoffman,Consumer
4,5,AA-10480,Andrew Allen,Consumer


In [93]:
df_original.loc[df_original['nome_consumidor'] == 'Darren Powers']dd

,id,id_ped,data_pedido,data_envio,modo_envio,id_cons,nome_consumidor,perfil_consumidor,pais,cidade,estado,codigo_postal,regiao,id_prod,categoria,sub_categoria,nome_produto,preco
49,50,CA-2016-115742,18/04/2016,22/04/2016,Standard Class,DP-13000,Darren Powers,Consumer,United States,New Albany,Indiana,47150.0,Central,OFF-BI-10004410,Office Supplies,Binders,"C-Line Peel & Stick Add-On Filing Pockets, 8-3...",38.220
50,51,CA-2016-115742,18/04/2016,22/04/2016,Standard Class,DP-13000,Darren Powers,Consumer,United States,New Albany,Indiana,47150.0,Central,OFF-LA-10002762,Office Supplies,Labels,Avery 485,75.180
51,52,CA-2016-115742,18/04/2016,22/04/2016,Standard Class,DP-13000,Darren Powers,Consumer,United States,New Albany,Indiana,47150.0,Central,FUR-FU-10001706,Furniture,Furnishings,Longer-Life Soft White Bulbs,6.160
52,53,CA-2016-115742,18/04/2016,22/04/2016,Standard Class,DP-13000,Darren Powers,Consumer,United States,New Albany,Indiana,47150.0,Central,FUR-CH-10003061,Furniture,Chairs,"Global Leather Task Chair, Black",89.990
1777,1778,CA-2016-105347,24/11/2016,28/11/2016,Standard Class,DP-13000,Darren Powers,Consumer,United States,Los Angeles,California,90004.0,West,OFF-PA-10000675,Office Supplies,Paper,Xerox 1919,368.910
1778,1779,CA-2016-105347,24/11/2016,28/11/2016,Standard Class,DP-13000,Darren Powers,Consumer,United States,Los Angeles,California,90004.0,West,OFF-AR-10003045,Office Supplies,Art,Prang Colored Pencils,14.700
4964,4965,CA-2017-141180,15/05/2017,19/05/2017,Second Class,DP-13000,Darren Powers,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-BI-10000301,Office Supplies,Binders,GBC Instant Report Kit,7.764
5295,5296,CA-2018-142174,04/03/2018,09/03/2018,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,Texas,77041.0,Central,OFF-PA-10000806,Office Supplies,Paper,Xerox 1934,89.568
6544,6545,US-2018-154872,14/05/2018,18/05/2018,Standard Class,DP-13000,Darren Powers,Consumer,United States,Cleveland,Ohio,44105.0,East,OFF-BI-10003007,Office Supplies,Binders,"Premium Transparent Presentation Covers, No Pa...",58.170
6545,6546,US-2018-154872,14/05/2018,18/05/2018,Standard Class,DP-13000,Darren Powers,Consumer,United States,Cleveland,Ohio,44105.0,East,OFF-LA-10000973,Office Supplies,Labels,Avery 502,5.040


In [61]:
df_consumidor2 = df_original.groupby('id_cons')
df_consumidor2.head()
# df_consumidor2.loc[df_consumidor2['nome_consumidor'] == 'Darren Powers']

,id,id_ped,data_pedido,data_envio,modo_envio,id_cons,nome_consumidor,perfil_consumidor,pais,cidade,estado,codigo_postal,regiao,id_prod,categoria,sub_categoria,nome_produto,preco
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9736,9737,US-2016-100069,29/06/2016,03/07/2016,Standard Class,NF-18475,Neil Französisch,Home Office,United States,Omaha,Nebraska,68104.0,Central,TEC-PH-10004667,Technology,Phones,Cisco 8x8 Inc. 6753i IP Business Phone System,269.9800
9737,9738,CA-2018-129294,16/03/2018,21/03/2018,Standard Class,KD-16615,Ken Dana,Corporate,United States,Los Angeles,California,90032.0,West,OFF-ST-10002615,Office Supplies,Storage,"Dual Level, Single-Width Filing Carts",310.1200
9738,9739,CA-2018-129294,16/03/2018,21/03/2018,Standard Class,KD-16615,Ken Dana,Corporate,United States,Los Angeles,California,90032.0,West,OFF-BI-10004236,Office Supplies,Binders,"XtraLife ClearVue Slant-D Ring Binder, White, 3""",70.4640
9744,9745,CA-2018-141782,21/01/2018,25/01/2018,Standard Class,BE-11410,Bobby Elias,Consumer,United States,Aurora,Illinois,60505.0,Central,OFF-EN-10002230,Office Supplies,Envelopes,Airmail Envelopes,268.5760


In [12]:
# df_pedido = df_original.select('id_ped', 'data_pedido', 'data_envio', 'modo_envio', 'pais', 'cidade', 'estado', 'codigo_postal', 'regiao', 'id_cons').withColumn('id', monotonically_increasing_id() + 1)

df_pedido = df_original[['id_ped', 'data_pedido', 'data_envio', 'modo_envio', 'pais', 'cidade', 'estado', 'codigo_postal', 'regiao', 'id_cons']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id'})
df_pedido['id'] = df_pedido['id'] + 1

df_pedido.head(5)

,id,id_ped,data_pedido,data_envio,modo_envio,pais,cidade,estado,codigo_postal,regiao,id_cons
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,United States,Henderson,Kentucky,42420.0,South,CG-12520
1,2,CA-2017-138688,12/06/2017,16/06/2017,Second Class,United States,Los Angeles,California,90036.0,West,DV-13045
2,3,US-2016-108966,11/10/2016,18/10/2016,Standard Class,United States,Fort Lauderdale,Florida,33311.0,South,SO-20335
3,4,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,United States,Los Angeles,California,90032.0,West,BH-11710
4,5,CA-2018-114412,15/04/2018,20/04/2018,Standard Class,United States,Concord,North Carolina,28027.0,South,AA-10480


#### Criando os relacionamentos entre as entidades

In [13]:
df_pedido['id_transporte'] = np.nan
df_pedido['id_endereco'] = np.nan
df_pedido['id_consumidor'] = np.nan

In [14]:
for index, row in df_transporte.iterrows():
    df_pedido.loc[df_pedido['modo_envio'] == row['titulo'], 'id_transporte'] = row['id']

df_pedido.head(5)

,id,id_ped,data_pedido,data_envio,modo_envio,pais,cidade,estado,codigo_postal,regiao,id_cons,id_transporte,id_endereco,id_consumidor
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,United States,Henderson,Kentucky,42420.0,South,CG-12520,1.0,NaN,NaN
1,2,CA-2017-138688,12/06/2017,16/06/2017,Second Class,United States,Los Angeles,California,90036.0,West,DV-13045,1.0,NaN,NaN
2,3,US-2016-108966,11/10/2016,18/10/2016,Standard Class,United States,Fort Lauderdale,Florida,33311.0,South,SO-20335,2.0,NaN,NaN
3,4,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,United States,Los Angeles,California,90032.0,West,BH-11710,2.0,NaN,NaN
4,5,CA-2018-114412,15/04/2018,20/04/2018,Standard Class,United States,Concord,North Carolina,28027.0,South,AA-10480,2.0,NaN,NaN


In [15]:
for index, row in df_endereco.iterrows():
    df_pedido.loc[df_pedido['codigo_postal'] == row['codigo_postal'], 'id_endereco'] = row['id']

df_pedido.head(5)

,id,id_ped,data_pedido,data_envio,modo_envio,pais,cidade,estado,codigo_postal,regiao,id_cons,id_transporte,id_endereco,id_consumidor
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,United States,Henderson,Kentucky,42420.0,South,CG-12520,1.0,1.0,NaN
1,2,CA-2017-138688,12/06/2017,16/06/2017,Second Class,United States,Los Angeles,California,90036.0,West,DV-13045,1.0,2.0,NaN
2,3,US-2016-108966,11/10/2016,18/10/2016,Standard Class,United States,Fort Lauderdale,Florida,33311.0,South,SO-20335,2.0,3.0,NaN
3,4,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,United States,Los Angeles,California,90032.0,West,BH-11710,2.0,4.0,NaN
4,5,CA-2018-114412,15/04/2018,20/04/2018,Standard Class,United States,Concord,North Carolina,28027.0,South,AA-10480,2.0,5.0,NaN


In [16]:
for index, row in df_consumidor.iterrows():
    df_pedido.loc[df_pedido['id_cons'] == row['id_cons'], 'id_consumidor'] = row['id']

df_pedido.head(5)

,id,id_ped,data_pedido,data_envio,modo_envio,pais,cidade,estado,codigo_postal,regiao,id_cons,id_transporte,id_endereco,id_consumidor
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,United States,Henderson,Kentucky,42420.0,South,CG-12520,1.0,1.0,1.0
1,2,CA-2017-138688,12/06/2017,16/06/2017,Second Class,United States,Los Angeles,California,90036.0,West,DV-13045,1.0,2.0,2.0
2,3,US-2016-108966,11/10/2016,18/10/2016,Standard Class,United States,Fort Lauderdale,Florida,33311.0,South,SO-20335,2.0,3.0,3.0
3,4,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,United States,Los Angeles,California,90032.0,West,BH-11710,2.0,4.0,4.0
4,5,CA-2018-114412,15/04/2018,20/04/2018,Standard Class,United States,Concord,North Carolina,28027.0,South,AA-10480,2.0,5.0,5.0


In [17]:
df_consumidor['id_perfil'] = np.nan

In [18]:
for index, row in df_perfil.iterrows():
    df_consumidor.loc[df_consumidor['perfil_consumidor'] == row['titulo'], 'id_perfil'] = row['id']

df_consumidor.head(5)

,id,id_cons,nome,perfil_consumidor,id_perfil
0,1,CG-12520,Claire Gute,Consumer,1.0
1,2,DV-13045,Darrin Van Huff,Corporate,2.0
2,3,SO-20335,Sean O'Donnell,Consumer,1.0
3,4,BH-11710,Brosina Hoffman,Consumer,1.0
4,5,AA-10480,Andrew Allen,Consumer,1.0


In [19]:
df_produto['id_sub_categoria'] = np.nan

In [20]:
for index, row in df_sub_categoria.iterrows():
    df_produto.loc[df_produto['sub_categoria'] == row['titulo'], 'id_sub_categoria'] = row['id']

df_produto.head(5)

,id,id_prod,nome,sub_categoria,id_sub_categoria
0,1,FUR-BO-10001798,Bush Somerset Collection Bookcase,Bookcases,1.0
1,2,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Chairs,2.0
2,3,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Labels,3.0
3,4,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Tables,4.0
4,5,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Storage,5.0


In [21]:
df_sub_categoria['id_categoria'] = np.nan

In [22]:
for index, row in df_categoria.iterrows():
    df_sub_categoria.loc[df_sub_categoria['categoria'] == row['titulo'], 'id_categoria'] = row['id']

df_sub_categoria.head(5)

,id,titulo,categoria,id_categoria
0,1,Bookcases,Furniture,1.0
1,2,Chairs,Furniture,1.0
2,3,Labels,Office Supplies,2.0
3,4,Tables,Furniture,1.0
4,5,Storage,Office Supplies,2.0


#### Criando relacionamentos da tabela associativa

In [23]:
df_assoc_pedido_produto = df_original[['id_ped', 'id_prod', 'preco']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index': 'id'})

df_assoc_pedido_produto['id_pedido'] = np.nan
df_assoc_pedido_produto['id_produto'] = np.nan

In [24]:
for index, row in df_pedido.iterrows():
    df_assoc_pedido_produto.loc[df_assoc_pedido_produto['id_ped'] == row['id_ped'], 'id_pedido'] = row['id']

In [25]:
for index, row in df_produto.iterrows():
    df_assoc_pedido_produto.loc[df_assoc_pedido_produto['id_prod'] == row['id_prod'], 'id_produto'] = row['id']

In [26]:
df_assoc_pedido_produto.head(5)

,id,id_ped,id_prod,preco,id_pedido,id_produto
0,0,CA-2017-152156,FUR-BO-10001798,261.9600,1.0,1.0
1,1,CA-2017-152156,FUR-CH-10000454,731.9400,1.0,2.0
2,2,CA-2017-138688,OFF-LA-10000240,14.6200,2.0,3.0
3,3,US-2016-108966,FUR-TA-10000577,957.5775,3.0,4.0
4,4,US-2016-108966,OFF-ST-10000760,22.3680,3.0,5.0


#### Traduzindo valores

In [27]:
modo_envio = {
    'Standard Class': 'Padrão',
    'Second Class': 'Segunda Classe',
    'First Class': 'Primeira Classe',
    'Same Day': 'No Mesmo Dia'
}

perfil_consumidor = {
    'Consumer': 'Consumidor',
    'Corporate': 'Corporativo',
    'Home Office': 'Home Office'
}

pais = {
    'United States': 'Estados Unidos'
}

regiao = {
    'West': 'Oeste',
    'East': 'Leste',
    'Central': 'Central',
    'South': 'Sul'
}

categoria = {
    'Office Supplies': 'Material de Escritório',
    'Furniture': 'Móveis',
    'Technology': 'Tecnologia'
}

sub_categoria = {
    'Envelopes': 'Envelopes',
    'Art': 'Arte',
    'Chairs': 'Cadeiras',
    'Furnishings': 'Mobiliário',
    'Supplies': 'Suprimentos',
    'Fasteners': 'Fechos',
    'Binders': 'Fichários',
    'Bookcases': 'Estantes',
    'Labels': 'Etiquetas',
    'Paper': 'Papel',
    'Accessories': 'Acessórios',
    'Copiers': 'Copiadoras',
    'Phones': 'Telefones',
    'Machines': 'Máquinas',
    'Storage': 'Armazenar',
    'Appliances': 'Eletrodomésticos',
    'Tables': 'Tabelas'
}

In [28]:
df_categoria['titulo'] = df_categoria['titulo'].map(categoria)

In [29]:
df_sub_categoria['titulo'] = df_sub_categoria['titulo'].map(sub_categoria)

In [30]:
df_transporte['titulo'] = df_transporte['titulo'].map(modo_envio)

In [31]:
df_perfil['titulo'] = df_perfil['titulo'].map(perfil_consumidor)

In [32]:
df_endereco['pais'] = df_endereco['pais'].map(pais)
df_endereco['regiao'] = df_endereco['regiao'].map(regiao)

#### Limpando as colunas para o banco de dados

In [ ]:
df_sub_categoria.drop(columns = ['categoria'], inplace = True)
df_sub_categoria.head(5)

,id,titulo,id_categoria
0,1,Estantes,1.0
1,2,Cadeiras,1.0
2,3,Etiquetas,2.0
3,4,Tabelas,1.0
4,5,Armazenar,2.0


In [ ]:
df_produto.drop(columns = ['sub_categoria', 'id_prod'], inplace = True)
df_produto.head(5)

,id,nome,id_sub_categoria
0,1,Bush Somerset Collection Bookcase,1.0
1,2,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",2.0
2,3,Self-Adhesive Address Labels for Typewriters b...,3.0
3,4,Bretford CR4500 Series Slim Rectangular Table,4.0
4,5,Eldon Fold 'N Roll Cart System,5.0


In [ ]:
df_assoc_pedido_produto.drop(columns = ['id_ped', 'id_prod'], inplace = True)
df_assoc_pedido_produto.head(5)

,id,preco,id_pedido,id_produto
0,0,261.9600,1.0,1.0
1,1,731.9400,1.0,2.0
2,2,14.6200,2.0,3.0
3,3,957.5775,3.0,4.0
4,4,22.3680,3.0,5.0


In [ ]:
df_pedido.drop(columns = ['id_ped', 'modo_envio', 'pais', 'cidade', 'estado', 'codigo_postal', 'regiao', 'id_cons'], inplace = True)
df_pedido.head(5)

,id,data_pedido,data_envio,id_transporte,id_endereco,id_consumidor
0,1,08/11/2017,11/11/2017,1.0,1.0,1.0
1,2,12/06/2017,16/06/2017,1.0,2.0,2.0
2,3,11/10/2016,18/10/2016,2.0,3.0,3.0
3,4,09/06/2015,14/06/2015,2.0,4.0,4.0
4,5,15/04/2018,20/04/2018,2.0,5.0,5.0


In [ ]:
df_consumidor.drop(columns = ['id_cons', 'perfil_consumidor'], inplace = True)
df_consumidor.head(5)

,id,nome,id_perfil
0,1,Claire Gute,1.0
1,2,Darrin Van Huff,2.0
2,3,Sean O'Donnell,1.0
3,4,Brosina Hoffman,1.0
4,5,Andrew Allen,1.0


#### Inserindo via script externo

In [ ]:
insert_categoria = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_categoria.sql', 'w', encoding = 'utf-8')
insert_categoria.write(f"INSERT INTO categoria (titulo) VALUES \n")

df_categoria.dropna(inplace = True)

for index, row in df_categoria.iterrows():
    insert_categoria.write(f"('{row['titulo']}'),\n")

insert_categoria.close()

In [ ]:
insert_sub_categoria = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_sub_categoria.sql', 'w', encoding = 'utf-8')
insert_sub_categoria.write(f"INSERT INTO subcategoria (titulo, id_categoria) VALUES \n")

df_sub_categoria.dropna(inplace = True)
df_sub_categoria['id_categoria'] = df_sub_categoria['id_categoria'].astype(int)

for index, row in df_sub_categoria.iterrows():
    insert_sub_categoria.write(f"('{row['titulo']}', {row['id_categoria']}),\n")

insert_sub_categoria.close()

In [ ]:
df_produto.rename(columns = {'id_sub_categoria': 'id_subcategoria'}, inplace = True)
df_produto['id_subcategoria'] = df_produto['id_subcategoria'].astype(int)
df_produto['nome'] = df_produto['nome'].str[:127]
df_produto['nome'] = df_produto['nome'].str.replace("'", "")

In [ ]:
insert_produto = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_produto.sql', 'w', encoding = 'utf-8')
insert_produto.write(f"INSERT INTO produto (nome, id_subcategoria) VALUES \n")

df_produto.dropna(inplace = True)

for index, row in df_produto.iterrows():
    insert_produto.write(f"('{row['nome']}', {row['id_subcategoria']}),\n")

insert_produto.close()

In [ ]:
insert_perfil = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_perfil.sql', 'w', encoding = 'utf-8')
insert_perfil.write(f"INSERT INTO perfil (titulo) VALUES \n")

df_perfil.dropna(inplace = True)

for index, row in df_perfil.iterrows():
    insert_perfil.write(f"('{row['titulo']}'),\n")

insert_perfil.close()

In [ ]:
df_consumidor['nome'] = df_consumidor['nome'].str.replace("'", "")
insert_consumidor = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_consumidor.sql', 'w', encoding = 'utf-8')
insert_consumidor.write(f"INSERT INTO consumidor (nome, id_perfil) VALUES \n")

df_consumidor.dropna(inplace = True)

for index, row in df_consumidor.iterrows():
    insert_consumidor.write(f"('{row['nome']}', {row['id_perfil']}),\n")

insert_consumidor.close()

In [ ]:
df_endereco['estado'] = df_endereco['estado'].str.replace("'", "")
df_endereco['cidade'] = df_endereco['cidade'].str.replace("'", "")
df_endereco.dropna(inplace = True)


df_endereco['codigo_postal'] = df_endereco['codigo_postal'].astype(int)

insert_endereco = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_endereco.sql', 'w', encoding = 'utf-8')
insert_endereco.write(f"INSERT INTO endereco (pais, cidade, estado, codigo_postal, regiao) VALUES \n")


for index, row in df_endereco.iterrows():
    insert_endereco.write(f"('{row['pais']}', '{row['cidade']}', '{row['estado']}', '{row['codigo_postal']}', '{row['regiao']}'),\n")

insert_endereco.close()

In [ ]:
insert_transportes = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_transportes.sql', 'w', encoding = 'utf-8')
insert_transportes.write(f"INSERT INTO transporte (id, titulo) VALUES \n")

df_transporte.dropna(inplace = True)

for index, row in df_transporte.iterrows():
    insert_transportes.write(f"('{row['titulo']}'),\n")

insert_transportes.close()

In [ ]:
insert_pedidos = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_pedidos.sql', 'w', encoding = 'utf-8')
insert_pedidos.write(f"INSERT INTO pedido (id, data_pedido, data_envio, id_transporte, id_endereco, id_consumidor) VALUES \n")

df_pedido.dropna(inplace = True)

df_pedido['id_transporte'] = df_pedido['id_transporte'].astype(int)
df_pedido['id_endereco'] = df_pedido['id_endereco'].astype(int)
df_pedido['id_consumidor'] = df_pedido['id_consumidor'].astype(int)

for index, row in df_pedido.iterrows():
    insert_pedidos.write(f"({row['id']}, '{datetime.strptime(row['data_pedido'], '%d/%m/%Y')}', '{datetime.strptime(row['data_envio'], '%d/%m/%Y')}', {row['id_transporte']}, {row['id_endereco']}, {row['id_consumidor']}),\n")

insert_pedidos.close()

In [ ]:
insert_df_assoc_pedido_produto = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_assoc_pedido_produto.sql', 'w', encoding = 'utf-8')
insert_df_assoc_pedido_produto.write(f"INSERT INTO produto_pedido (id_pedido, id_produto, preco) VALUES \n")

df_assoc_pedido_produto['id_pedido'] = df_assoc_pedido_produto['id_pedido'].astype(int)
df_assoc_pedido_produto['id_produto'] = df_assoc_pedido_produto['id_produto'].astype(int)

df_assoc_pedido_produto.dropna(inplace = True)

for index, row in df_assoc_pedido_produto.iterrows():
    insert_df_assoc_pedido_produto.write(f"({row['id_pedido']}, {row['id_produto']}, {row['preco']}),\n")

insert_df_assoc_pedido_produto.close()

#### Juntando tudo dentro de um Script de DML

In [ ]:
script = open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/script.sql', 'w', encoding = 'utf-8')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_categoria.sql', 'r', encoding = 'utf-8'):
    script.write(linha)
script.write('\n')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_sub_categoria.sql', 'r', encoding = 'utf-8'):
    script.write(linha)
script.write('\n')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_produto.sql', 'r', encoding = 'utf-8'):
    script.write(linha)
script.write('\n')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_perfil.sql', 'r', encoding = 'utf-8'):
    script.write(linha)
script.write('\n')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_consumidor.sql', 'r', encoding = 'utf-8'):
    script.write(linha)
script.write('\n')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_endereco.sql', 'r', encoding = 'utf-8'):
    script.write(linha)
script.write('\n')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_transportes.sql', 'r', encoding = 'utf-8'):
    script.write(linha)
script.write('\n')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_pedidos.sql', 'r', encoding = 'utf-8'): 
    script.write(linha)
script.write('\n')

for linha in open('C:/Users/leonardo/OneDrive/Documentos/supermarket-sales-data/sql/scripts/dml/apoio/insert_assoc_pedido_produto.sql', 'r', encoding = 'utf-8'):
    script.write(linha)
script.write('\n')

script.close()